In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/earthquakes/consolidated_data.csv
/kaggle/input/earthquakes/consolidated_data/consolidated_data.csv


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 13.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=9d04612e6564fcf0a9cb67acf9e00c737bacba33314f8cd768f47ccf6b85113d
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/16 15:18:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 01 : DataUnderstanding
The objective of the present Task is to perform Data Reading, Description and Understanding of the present Dataset.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SQLContext, SparkSession

In [6]:
path = '/kaggle/input/earthquakes/consolidated_data.csv'
#df2 = spark.read.option("delimiter", ",").csv(path)
#df2.show()

In [7]:
df = spark.read.option("escape","\"").csv(path, header='true', inferSchema='true')

In [8]:
df.show()

22/12/16 15:18:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
 Schema: _c0, time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
Expected: _c0 but found: 
CSV file: file:///kaggle/input/earthquakes/consolidated_data.csv
+------+--------------------+------------------+-------------------+-----+----+-------+----+-----+-------------------+----+------+------------+--------------------+--------------------+------------+-------------------+----------+-------------------+------+---------+--------------+---------+
|   _c0|                time|          latitude|          longitude|depth| mag|magType| nst|  gap|               dmin| rms|   net|          id|   

In [9]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- depth: double (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- nst: double (nullable = true)
 |-- gap: double (nullable = true)
 |-- dmin: double (nullable = true)
 |-- rms: double (nullable = true)
 |-- net: string (nullable = true)
 |-- id: string (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- place: string (nullable = true)
 |-- type: string (nullable = true)
 |-- horizontalError: double (nullable = true)
 |-- depthError: double (nullable = true)
 |-- magError: double (nullable = true)
 |-- magNst: double (nullable = true)
 |-- status: string (nullable = true)
 |-- locationSource: string (nullable = true)
 |-- magSource: string (nullable = true)



# Objectives
1. Get a general Idea of the Dataset and its features
2. Spot Missing values
3. Spot eventual mistakes/outliers 

# 1. Get a general Idea of the Dataset and its features

In [10]:
n_rows = df.count()

In [11]:
n_cols = len(df.columns)

In [12]:
print(f'Dimension of the Dataframe is: {(n_rows,n_cols)}')

Dimension of the Dataframe is: (3272774, 23)


In [13]:
df.columns

['_c0',
 'time',
 'latitude',
 'longitude',
 'depth',
 'mag',
 'magType',
 'nst',
 'gap',
 'dmin',
 'rms',
 'net',
 'id',
 'updated',
 'place',
 'type',
 'horizontalError',
 'depthError',
 'magError',
 'magNst',
 'status',
 'locationSource',
 'magSource']

In [14]:
# Get more information about the meaning of the columns 
#https://earthquake.usgs.gov/earthquakes/feed/v1.0/csv.php

In [15]:
# first create a copy of df 
df_copy = df.alias('df_copy')

In [16]:
# We want to drop duplicates 
df = df.dropDuplicates([col for col in df.columns if col != '_c0'])

In [17]:
# NB: this is the new length of the df after dropping duplicates
df.count()

3256955

The number of records is 3.256.955, which is the same as the number of distinct records on the column _c0. 
Considering the above statement and taking into consideration the fact that 'id' is a string providing the following information : " *'id': A unique identifier for the event. This is the current preferred id for the event, and may change over time. See the "ids" GeoJSON format property.* ", 
we decide to remove the following columns:

In [18]:
# df id is a string and does not provide any important information
df.select('id').take(5)

[Row(id='usp00003ku'),
 Row(id='usp00017cz'),
 Row(id='usp00050hn'),
 Row(id='usp0005273'),
 Row(id='usp00008pp')]

In [19]:
# First drop the following columns which are not useful for sure
to_drop = ['locationSource', 'id',
 'magSource', 'updated']

In [20]:
df = df.drop(*(col for col in to_drop))

In [21]:
df.columns

['_c0',
 'time',
 'latitude',
 'longitude',
 'depth',
 'mag',
 'magType',
 'nst',
 'gap',
 'dmin',
 'rms',
 'net',
 'place',
 'type',
 'horizontalError',
 'depthError',
 'magError',
 'magNst',
 'status']

# 2: Spot Missing values + spot eventual mistakes/outliers

In [22]:
# NaNs
from pyspark.sql.functions import col,sum
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()

22/12/16 15:20:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
 Schema: _c0, time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
Expected: _c0 but found: 
CSV file: file:///kaggle/input/earthquakes/consolidated_data.csv


22/12/16 15:20:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:24 WARN RowBasedKeyValueBatch: Calling spill() on

+---+----+--------+---------+-----+------+-------+------+------+-------+------+---+-----+----+---------------+----------+--------+------+------+
|_c0|time|latitude|longitude|depth|   mag|magType|   nst|   gap|   dmin|   rms|net|place|type|horizontalError|depthError|magError|magNst|status|
+---+----+--------+---------+-----+------+-------+------+------+-------+------+---+-----+----+---------------+----------+--------+------+------+
|  0|   0|       0|        0|    9|155698| 166609|876988|834294|1339697|210659|  0|   11|   0|        1524519|    603806| 1772000|983988|     1|
+---+----+--------+---------+-----+------+-------+------+------+-------+------+---+-----+----+---------------+----------+--------+------+------+



In [23]:
# remove all the rows presenting Nans on "mag" and "magType"
df = df.na.drop(subset=["mag", "magType"])

In [24]:
# delete rows presenting a few Nans 
to_drop = ['depth']
df = df.na.drop(subset = to_drop)

In [25]:
# for status and we directly delete the columns (not so informtive)
df = df.drop( 'place','status')

In [26]:
# Let's look at the distinct types of columns, so as to have an idea of the percentages
#df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()
from pyspark.sql.functions import *
import pyspark.sql.functions as F
#F.round((F.col('Month_start')/F.col('Month_end'))*100,2))
perc = df.select(*(F.round(((sum(col(c).isNull().cast("int")))/count(lit(1))*100),2).alias(c) for c in df.columns))
perc.show()

22/12/16 15:20:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
 Schema: _c0, time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
Expected: _c0 but found: 
CSV file: file:///kaggle/input/earthquakes/consolidated_data.csv


22/12/16 15:20:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:20:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:04 WARN RowBasedKeyValueBatch: Calling spill() on

+---+----+--------+---------+-----+---+-------+----+-----+-----+----+---+----+---------------+----------+--------+------+
|_c0|time|latitude|longitude|depth|mag|magType| nst|  gap| dmin| rms|net|type|horizontalError|depthError|magError|magNst|
+---+----+--------+---------+-----+---+-------+----+-----+-----+----+---+----+---------------+----------+--------+------+
|0.0| 0.0|     0.0|      0.0|  0.0|0.0|    0.0|26.3|24.66|39.48|6.17|0.0| 0.0|          45.03|     16.74|   51.95| 26.45|
+---+----+--------+---------+-----+---+-------+----+-----+-----+----+---+----+---------------+----------+--------+------+



#### Current situation
**nst** --> 26%
**gap** --> 23%
**dmin** --> 40%
**rms** --> 6%
**horizontalError** --> 45%
**depthError** --> 11%
**magError** --> 52%
**magNst** --> 22%

In [27]:
#Remove columns presenting too many missing values: dmin, horizontalError, magError
df = df.drop('dmin', 'horizontalError', 'magError')
#Remove missing values on rms (only 6%)
df = df.na.drop(subset=['rms'])

In [28]:
perc = df.select(*(F.round(((sum(col(c).isNull().cast("int")))/count(lit(1))*100),2).alias(c) for c in df.columns))
perc.show()

22/12/16 15:21:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
 Schema: _c0, time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
Expected: _c0 but found: 
CSV file: file:///kaggle/input/earthquakes/consolidated_data.csv


22/12/16 15:21:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:21:41 WARN RowBasedKeyValueBatch: Calling spill() on

+---+----+--------+---------+-----+---+-------+-----+-----+---+---+----+----------+------+
|_c0|time|latitude|longitude|depth|mag|magType|  nst|  gap|rms|net|type|depthError|magNst|
+---+----+--------+---------+-----+---+-------+-----+-----+---+---+----+----------+------+
|0.0| 0.0|     0.0|      0.0|  0.0|0.0|    0.0|25.67|23.01|0.0|0.0| 0.0|     11.27| 22.01|
+---+----+--------+---------+-----+---+-------+-----+-----+---+---+----+----------+------+



In [29]:
df.count()

2899664

# 3: Spot Errors

In [30]:
def stats(df):
    for i in df.dtypes:
        if(i[1] == 'int' or i[1] == 'double'):
            df.describe(i[0]).show(truncate=False)

In [31]:
stats(df)

22/12/16 15:22:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
 Schema: _c0, time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
Expected: _c0 but found: 
CSV file: file:///kaggle/input/earthquakes/consolidated_data.csv


22/12/16 15:22:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:22:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:22:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:22:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:22:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:22:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:22:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:22:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:22:45 WARN RowBasedKeyValueBatch: Calling spill() on

+-------+------------------+
|summary|_c0               |
+-------+------------------+
|count  |2899664           |
|mean   |1631868.2860765937|
|stddev |945401.3232190389 |
|min    |0                 |
|max    |3272773           |
+-------+------------------+



+-------+------------------+
|summary|latitude          |
+-------+------------------+
|count  |2899664           |
|mean   |37.10635068675379 |
|stddev |18.509435937673196|
|min    |-84.42200000000001|
|max    |87.221            |
+-------+------------------+



+-------+-------------------+
|summary|longitude          |
+-------+-------------------+
|count  |2899664            |
|mean   |-99.23720517124138 |
|stddev |76.03691288465575  |
|min    |-179.99900000000002|
|max    |180.0              |
+-------+-------------------+



...

+-------+------------------+
|summary|nst               |
+-------+------------------+
|count  |2155437           |
|mean   |15.989824801188808|
|stddev |26.720432399388894|
|min    |0.0               |
|max    |934.0             |
+-------+------------------+



+-------+------------------+
|summary|gap               |
+-------+------------------+
|count  |2232441           |
|mean   |126.67369488862093|
|stddev |67.42756857368224 |
|min    |0.0               |
|max    |360.0             |
+-------+------------------+



+-------+-------------------+
|summary|rms                |
+-------+-------------------+
|count  |2899664            |
|mean   |0.30964346338056215|
|stddev |0.38575488697807125|
|min    |-1.0               |
|max    |104.33             |
+-------+-------------------+



+-------+------------------+
|summary|depthError        |
+-------+------------------+
|count  |2572770           |
|mean   |5.6276865655315325|
|stddev |1188.7877753397745|
|min    |-1.0              |
|max    |1773552.5         |
+-------+------------------+



+-------+------------------+
|summary|magNst            |
+-------+------------------+
|count  |2261327           |
|mean   |12.594240019245337|
|stddev |21.065921641663053|
|min    |0.0               |
|max    |941.0             |
+-------+------------------+



## Considerations: 
- **depth**: some values are around -10 and this is an error for sure: drop such values.
- **mag** : https://it.wikipedia.org/wiki/Scala_Richter http://www.blueplanetheart.it/2018/12/perche-esistono-la-magnitudo-0-0-la-magnitudo-negativa/ some values can be negative, but they are not proper sismic waves. Some values could also be caused by events for which the magnitudo is quite low. Let's make a groupby
- **nst**: some values are 0, but they are many values. Keep them.
- **gap**: Typical Values
[0.0, 180.0]
Description:
The largest azimuthal gap between azimuthally adjacent stations (in degrees). In general, the smaller this number, the more reliable is the calculated horizontal position of the earthquake. Earthquake locations in which the azimuthal gap exceeds 180 degrees typically have large location and depth uncertainties.
- **rms**: Typical Values
[0.13,1.39]
Description
The root-mean-square (RMS) travel time residual, in sec, using all weights. This parameter provides a measure of the fit of the observed arrival times to the predicted arrival times for this location. Smaller numbers reflect a better fit of the data. The value is dependent on the accuracy of the velocity model used to compute the earthquake location, the quality weights assigned to the arrival time data, and the procedure used to locate the earthquake.
- **depthError**: Typical Values
[0, 100]
Description
Uncertainty of reported depth of the event in kilometers.
Additional Information
The depth error, in km, defined as the largest projection of the three principal errors on a vertical line.
- **magNst**:  same for nst

In [32]:
#MAG
df.groupBy("mag","type").count().orderBy(col("mag").asc()).show(150,truncate=False)

22/12/16 15:28:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:28:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:28:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:28:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:28:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:28:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:28:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:28:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-----+------------+-----+
|mag  |type        |count|
+-----+------------+-----+
|-9.99|earthquake  |670  |
|-5.0 |other event |1    |
|-3.0 |earthquake  |1    |
|-2.6 |earthquake  |2    |
|-2.5 |earthquake  |4    |
|-2.2 |earthquake  |1    |
|-2.0 |earthquake  |4    |
|-1.9 |earthquake  |5    |
|-1.8 |earthquake  |5    |
|-1.76|earthquake  |1    |
|-1.7 |earthquake  |4    |
|-1.6 |earthquake  |105  |
|-1.56|earthquake  |1    |
|-1.51|earthquake  |1    |
|-1.5 |earthquake  |18   |
|-1.4 |earthquake  |36   |
|-1.39|earthquake  |3    |
|-1.33|earthquake  |1    |
|-1.31|earthquake  |1    |
|-1.3 |earthquake  |142  |
|-1.29|earthquake  |2    |
|-1.28|earthquake  |1    |
|-1.27|earthquake  |1    |
|-1.24|earthquake  |3    |
|-1.23|earthquake  |3    |
|-1.22|earthquake  |1    |
|-1.21|earthquake  |2    |
|-1.2 |earthquake  |85   |
|-1.19|earthquake  |3    |
|-1.18|earthquake  |1    |
|-1.17|earthquake  |4    |
|-1.16|earthquake  |1    |
|-1.15|earthquake  |2    |
|-1.14|earthquake  |1    |
|

In [33]:
#NST
df.createOrReplaceTempView("dff")
spark.sql("select time,latitude,longitude,type from dff where nst = 0.0").show()

+--------------------+------------------+-------------------+------------+
|                time|          latitude|          longitude|        type|
+--------------------+------------------+-------------------+------------+
|1971-02-09 16:41:...|            34.416|            -118.37|  earthquake|
|1975-01-25 15:02:...|            32.995|           -115.505|  earthquake|
|1975-02-27 16:43:...|            34.427|           -118.406|  earthquake|
|1975-04-02 22:33:...|            33.548|-116.52799999999999|  earthquake|
|1975-04-21 15:41:...|            34.486|            -116.45|  earthquake|
|1975-06-01 02:01:...|            34.509|           -116.479|  earthquake|
|1975-06-17 16:03:...|             32.79|           -115.449|  earthquake|
|1975-06-20 23:26:...|            32.775|-115.43799999999999|  earthquake|
|1975-06-23 04:27:...|            32.893|           -115.523|  earthquake|
|1975-07-21 16:51:...|            33.729|           -116.846|  earthquake|
|1975-07-25 14:11:...|   

In [34]:
#NST
df.groupBy("nst").count().orderBy(col("nst").asc()).show(50,truncate=False)

22/12/16 15:29:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:29:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:29:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:29:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:29:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:29:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:29:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:29:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+----+------+
|nst |count |
+----+------+
|null|744227|
|0.0 |283801|
|1.0 |40    |
|2.0 |1836  |
|3.0 |27409 |
|4.0 |88082 |
|5.0 |119494|
|6.0 |129435|
|7.0 |132318|
|8.0 |122438|
|9.0 |110796|
|10.0|98898 |
|11.0|88487 |
|12.0|78436 |
|13.0|70012 |
|14.0|63546 |
|15.0|56900 |
|16.0|52065 |
|17.0|48070 |
|18.0|43433 |
|19.0|39480 |
|20.0|36193 |
|21.0|32943 |
|22.0|30181 |
|23.0|27427 |
|24.0|24750 |
|25.0|22660 |
|26.0|20774 |
|27.0|19048 |
|28.0|17462 |
|29.0|16044 |
|30.0|14848 |
|31.0|14168 |
|32.0|12767 |
|33.0|11539 |
|34.0|10840 |
|35.0|10177 |
|36.0|9412  |
|37.0|8868  |
|38.0|8165  |
|39.0|7891  |
|40.0|7328  |
|41.0|6874  |
|42.0|6420  |
|43.0|6065  |
|44.0|5583  |
|45.0|5322  |
|46.0|5043  |
|47.0|4613  |
|48.0|4305  |
+----+------+
only showing top 50 rows



In [35]:
#GAP
df.createOrReplaceTempView("dff")
spark.sql("select count(*) from dff where gap > 180").show()

+--------+
|count(1)|
+--------+
|  435377|
+--------+



In [36]:
#RMS --> remember this is a temporal feature: drop negarive values
df.createOrReplaceTempView("dff")
spark.sql("select count(*) from dff where rms < 0").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [37]:
#RMS
df.createOrReplaceTempView("dff")
spark.sql("select rms,count(*) from dff where rms > 10 group by rms order by rms asc").show(150)

+------------------+--------+
|               rms|count(1)|
+------------------+--------+
|             10.07|       1|
|             10.22|       1|
|             10.26|       1|
|            10.261|       1|
|             10.32|       1|
|             10.39|       1|
|10.562999999999999|       1|
|             10.59|       1|
|            10.659|       1|
|             10.72|       1|
|             10.94|       1|
|             10.96|       1|
|              11.0|       1|
|            11.019|       1|
|             11.02|       1|
|             11.08|       1|
|              11.1|       1|
|            11.155|       1|
|             11.23|       1|
|             11.38|       1|
|            11.413|       1|
|             11.54|       1|
|             11.57|       1|
|            11.741|       1|
|             11.95|       1|
|            11.954|       1|
|             12.02|       1|
|             12.09|       1|
|            12.169|       1|
|              12.4|       1|
|         

In [38]:
#DEPTHERROR : cannot be negative
df.createOrReplaceTempView("dff")
spark.sql("select count(*) from dff where depthError < 0").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [39]:
#DEPTHERROR 
df.createOrReplaceTempView("dff")
spark.sql("select count(*) from dff where depthError > 600").show()

+--------+
|count(1)|
+--------+
|     444|
+--------+



In [40]:
#DEPTHERROR 
df.createOrReplaceTempView("dff")
spark.sql("select count(*) from dff where depthError > 1000").show()

+--------+
|count(1)|
+--------+
|     291|
+--------+



In [41]:
#MAGNST
df.createOrReplaceTempView("dff")
spark.sql("select count(*) from dff where magNst = 0.0").show()

+--------+
|count(1)|
+--------+
|  106598|
+--------+



### FINAL DF

In [42]:
df = df.filter((df.depth > -6) & (df.mag >= -1) & (df.rms >= 0) &(df.depthError >= 0) & (df.depthError <= 500))  #totake
df.show()

22/12/16 15:30:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
 Schema: _c0, time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
Expected: _c0 but found: 
CSV file: file:///kaggle/input/earthquakes/consolidated_data.csv


22/12/16 15:30:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:30:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:30:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:30:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:30:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:31:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------+--------------------+------------------+-------------------+--------------------+----+-------+----+-----+----+---+----------+----------+------+
|    _c0|                time|          latitude|          longitude|               depth| mag|magType| nst|  gap| rms|net|      type|depthError|magNst|
+-------+--------------------+------------------+-------------------+--------------------+----+-------+----+-----+----+---+----------+----------+------+
| 988433|1970-01-12 08:42:...|           33.1405|-116.16766670000001|                 6.0|2.42|     mh| 6.0|129.0|0.53| ci|earthquake|     31.61|   9.0|
|1560841|1970-04-20 07:31:...|        34.4058333|           -118.557|                 1.5|2.65|     ml| 9.0|149.0|0.47| ci|earthquake|       2.1|   7.0|
| 774426|1970-09-04 05:57:...| 46.68600000000001|            -119.61|               4.555| 1.2|     md| 6.0|123.0|0.32| uw|earthquake|      9.09|   5.0|
| 774375|1970-09-15 22:30:...|        34.2436667|-117.50083329999998|             

In [43]:
df.count()

2570408

In [44]:
df.repartition(1).write.option("header",True).csv("DDAM_Pyspark.csv")

22/12/16 15:31:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
 Schema: _c0, time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource
Expected: _c0 but found: 
CSV file: file:///kaggle/input/earthquakes/consolidated_data.csv


22/12/16 15:31:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:31:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:31:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:31:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:31:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/16 15:32:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
